# LFM (Liquid Foundation Models) Demo

This notebook demonstrates how to use the Liquid Foundation Models (LFM) for text generation tasks.

## Features
- Load pre-trained LFM models
- Generate text with different sampling strategies
- Fine-tune models on custom data
- Visualize model outputs

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install torch transformers accelerate sentencepiece protobuf
!pip install matplotlib seaborn pandas numpy

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from typing import List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Load LFM Model

In [ ]:
# Model selection
model_options = {
    "1B": "liquid/lfm-1b",
    "3B": "liquid/lfm-3b", 
    "7B": "liquid/lfm-7b",
    "13B": "liquid/lfm-13b",
    "40B": "liquid/lfm-40b"
}

# Select model size based on available memory
model_size = "3B"  # Change this based on your hardware
model_name = model_options[model_size]

print(f"Loading {model_name}...")

In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

if not torch.cuda.is_available():
    model = model.to(device)

print(f"Model loaded successfully!")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B")

## 3. Text Generation Examples

In [ ]:
def generate_text(prompt: str, 
                  max_length: int = 100,
                  temperature: float = 0.7,
                  top_p: float = 0.9,
                  top_k: int = 50,
                  num_return_sequences: int = 1) -> List[str]:
    """
    Generate text using the LFM model.
    
    Args:
        prompt: Input text prompt
        max_length: Maximum length of generated text
        temperature: Sampling temperature (0.0 to 1.0)
        top_p: Nucleus sampling parameter
        top_k: Top-k sampling parameter
        num_return_sequences: Number of sequences to generate
    
    Returns:
        List of generated text sequences
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_texts = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        generated_texts.append(text)
    
    return generated_texts

In [ ]:
# Example prompts
prompts = [
    "The future of artificial intelligence is",
    "Once upon a time in a digital world",
    "The most important scientific discovery of the 21st century is",
    "To solve climate change, we need to"
]

# Generate text for each prompt
for prompt in prompts:
    print(f"\n{'='*80}")
    print(f"Prompt: {prompt}")
    print(f"{'='*80}")
    
    generated = generate_text(prompt, max_length=150, temperature=0.8)
    print(generated[0])

## 4. Interactive Text Generation

In [ ]:
# Interactive generation with parameter controls
from ipywidgets import interact, widgets

@interact(
    prompt=widgets.Textarea(value='The key to happiness is', description='Prompt:'),
    max_length=widgets.IntSlider(min=50, max=500, step=50, value=150, description='Max Length:'),
    temperature=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.7, description='Temperature:'),
    top_p=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=0.9, description='Top-p:'),
    top_k=widgets.IntSlider(min=10, max=100, step=10, value=50, description='Top-k:')
)
def interactive_generate(prompt, max_length, temperature, top_p, top_k):
    result = generate_text(prompt, max_length, temperature, top_p, top_k)[0]
    print(result)

## 5. Batch Processing and Analysis

In [ ]:
# Analyze generation with different temperatures
test_prompt = "The meaning of life is"
temperatures = [0.3, 0.5, 0.7, 0.9, 1.2]
results = []

for temp in temperatures:
    generated = generate_text(test_prompt, max_length=100, temperature=temp, num_return_sequences=3)
    results.append({
        'temperature': temp,
        'generations': generated
    })

# Display results
for result in results:
    print(f"\nTemperature: {result['temperature']}")
    print("-" * 50)
    for i, gen in enumerate(result['generations']):
        print(f"Sample {i+1}: {gen[:100]}...")

## 6. Visualization of Model Outputs

In [ ]:
# Analyze token probabilities
def get_token_probabilities(prompt: str, next_tokens: int = 10):
    """
    Get probabilities for the next tokens after a prompt.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[0, -1, :]
        probs = torch.softmax(logits, dim=-1)
        
    # Get top k tokens and their probabilities
    top_probs, top_indices = torch.topk(probs, next_tokens)
    top_tokens = [tokenizer.decode([idx]) for idx in top_indices]
    
    return top_tokens, top_probs.cpu().numpy()

# Visualize token probabilities
prompt = "The weather today is"
tokens, probs = get_token_probabilities(prompt, next_tokens=15)

plt.figure(figsize=(10, 6))
plt.bar(range(len(tokens)), probs)
plt.xticks(range(len(tokens)), tokens, rotation=45, ha='right')
plt.xlabel('Next Token')
plt.ylabel('Probability')
plt.title(f'Token Probabilities after: "{prompt}"')
plt.tight_layout()
plt.show()

In [ ]:
# Compare different sampling strategies
strategies = [
    {"name": "Greedy", "temperature": 0.1, "top_p": 1.0, "top_k": 1},
    {"name": "Low Temperature", "temperature": 0.5, "top_p": 0.9, "top_k": 50},
    {"name": "Medium Temperature", "temperature": 0.8, "top_p": 0.9, "top_k": 50},
    {"name": "High Temperature", "temperature": 1.2, "top_p": 0.95, "top_k": 100},
    {"name": "Nucleus Sampling", "temperature": 0.7, "top_p": 0.7, "top_k": 0}
]

prompt = "In the year 2050, technology will"
strategy_results = []

for strategy in strategies:
    generated = generate_text(
        prompt, 
        max_length=100,
        temperature=strategy["temperature"],
        top_p=strategy["top_p"],
        top_k=strategy["top_k"] if strategy["top_k"] > 0 else 50000
    )[0]
    
    strategy_results.append({
        "Strategy": strategy["name"],
        "Generated Text": generated[len(prompt):].strip()[:100] + "..."
    })

# Display as DataFrame
df = pd.DataFrame(strategy_results)
df

## 7. Fine-tuning Example (Optional)

In [ ]:
# Example of preparing data for fine-tuning
# Note: Actual fine-tuning requires more setup and computational resources

# Sample training data
training_examples = [
    {"input": "Translate to French: Hello, how are you?", "output": "Bonjour, comment allez-vous?"},
    {"input": "Summarize: The quick brown fox jumps over the lazy dog.", "output": "A fox jumps over a dog."},
    {"input": "Question: What is the capital of France?", "output": "Answer: Paris"},
]

# Format for fine-tuning
formatted_data = []
for example in training_examples:
    text = f"{example['input']}\n{example['output']}"
    formatted_data.append(text)

print("Sample formatted training data:")
for i, data in enumerate(formatted_data[:3]):
    print(f"\nExample {i+1}:")
    print(data)

## 8. Model Performance Metrics

In [ ]:
# Measure generation speed
import time

def measure_generation_speed(prompt: str, max_length: int = 100, num_runs: int = 5):
    times = []
    
    for _ in range(num_runs):
        start_time = time.time()
        _ = generate_text(prompt, max_length=max_length)
        end_time = time.time()
        times.append(end_time - start_time)
    
    avg_time = np.mean(times)
    tokens_per_second = max_length / avg_time
    
    return {
        "average_time": avg_time,
        "tokens_per_second": tokens_per_second,
        "all_times": times
    }

# Test generation speed
speed_results = measure_generation_speed("Once upon a time", max_length=100)

print(f"Average generation time: {speed_results['average_time']:.2f} seconds")
print(f"Tokens per second: {speed_results['tokens_per_second']:.2f}")

# Visualize generation times
plt.figure(figsize=(8, 4))
plt.plot(speed_results['all_times'], marker='o')
plt.axhline(y=speed_results['average_time'], color='r', linestyle='--', label='Average')
plt.xlabel('Run Number')
plt.ylabel('Time (seconds)')
plt.title('Generation Time per Run')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 9. Advanced Usage: Custom Generation

In [ ]:
class ConstrainedGenerator:
    """
    Generate text with custom constraints.
    """
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
    
    def generate_with_keywords(self, prompt: str, keywords: List[str], max_length: int = 100):
        """
        Generate text that includes specific keywords.
        """
        # This is a simplified example - real implementation would be more complex
        base_generation = generate_text(prompt, max_length=max_length)[0]
        
        # Check which keywords are missing
        missing_keywords = [kw for kw in keywords if kw.lower() not in base_generation.lower()]
        
        return {
            "generated_text": base_generation,
            "included_keywords": [kw for kw in keywords if kw.lower() in base_generation.lower()],
            "missing_keywords": missing_keywords
        }

# Example usage
generator = ConstrainedGenerator(model, tokenizer)
result = generator.generate_with_keywords(
    "Write a story about",
    keywords=["dragon", "castle", "knight"],
    max_length=200
)

print("Generated Text:")
print(result["generated_text"])
print(f"\nIncluded keywords: {result['included_keywords']}")
print(f"Missing keywords: {result['missing_keywords']}")

## 10. Save and Load Generated Content

In [ ]:
# Save generation history
import json
from datetime import datetime

generation_history = []

def save_generation(prompt: str, generated_text: str, parameters: dict):
    """
    Save generation to history.
    """
    entry = {
        "timestamp": datetime.now().isoformat(),
        "prompt": prompt,
        "generated_text": generated_text,
        "parameters": parameters
    }
    generation_history.append(entry)

# Generate and save some examples
test_prompts = [
    "The secret to success is",
    "In a world where AI can",
    "The most beautiful thing about nature is"
]

for prompt in test_prompts:
    params = {"temperature": 0.7, "max_length": 100, "top_p": 0.9}
    generated = generate_text(prompt, **params)[0]
    save_generation(prompt, generated, params)

# Display history
print(f"Generated {len(generation_history)} texts")
for i, entry in enumerate(generation_history):
    print(f"\n{i+1}. {entry['prompt'][:50]}...")
    print(f"   Generated: {entry['generated_text'][:100]}...")

In [ ]:
# Export history to file
with open('generation_history.json', 'w') as f:
    json.dump(generation_history, f, indent=2)

print("Generation history saved to 'generation_history.json'")

# Create a summary DataFrame
df_history = pd.DataFrame([
    {
        "Prompt": entry["prompt"][:30] + "...",
        "Temperature": entry["parameters"]["temperature"],
        "Length": len(entry["generated_text"].split()),
        "Timestamp": entry["timestamp"][:19]
    }
    for entry in generation_history
])

df_history

## Conclusion

This notebook demonstrated:
- Loading and using LFM models
- Various text generation strategies
- Interactive generation with parameter controls
- Performance analysis and visualization
- Custom generation constraints

Feel free to experiment with different prompts, parameters, and model sizes!